In [1]:
import pandas as pd
import requests
import json

In [2]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [3]:
req = requests.get('https://api.sncf.com/v1/coverage/sncf/disruptions//?since=20230201T000000&',auth=(token, ''))

In [4]:
doc = json.loads(req.text)
row = len(doc['disruptions'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 25


# Retard

## Liste des retards

In [58]:
df = pd.DataFrame(doc['disruptions'])
df.head(5)

,status,disruption_id,severity,impact_id,application_periods,updated_at,uri,impacted_objects,disruption_uri,contributor,cause,id,messages
0,past,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,"{'color': '#000000', 'priority': 42, 'name': '...",43e7b9a8-9622-4bb5-a46c-abaed55ef80d,"[{'begin': '20230227T001000', 'end': '20230227...",20230228T170251,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,[{'impacted_stops': [{'amended_arrival_time': ...,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,realtime.sncf.piv,,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,NaN
1,past,02bfacd7-1e3c-445a-ad1c-c2b48f121965,"{'color': '#000000', 'priority': 42, 'name': '...",02bfacd7-1e3c-445a-ad1c-c2b48f121965,"[{'begin': '20230226T010000', 'end': '20230226...",20230228T170251,02bfacd7-1e3c-445a-ad1c-c2b48f121965,[{'impacted_stops': [{'amended_arrival_time': ...,02bfacd7-1e3c-445a-ad1c-c2b48f121965,realtime.sncf.piv,,02bfacd7-1e3c-445a-ad1c-c2b48f121965,NaN
2,past,a17dec36-dc38-4b7c-998b-0c21622a45f0,"{'color': '#000000', 'priority': 42, 'name': '...",a17dec36-dc38-4b7c-998b-0c21622a45f0,"[{'begin': '20230228T130500', 'end': '20230228...",20230228T170251,a17dec36-dc38-4b7c-998b-0c21622a45f0,[{'impacted_stops': [{'amended_arrival_time': ...,a17dec36-dc38-4b7c-998b-0c21622a45f0,realtime.sncf.piv,,a17dec36-dc38-4b7c-998b-0c21622a45f0,"[{'text': 'Embouteillage', 'channel': {'conten..."
3,past,515a9899-e3d3-47b8-a336-483b2c7cbb89,"{'color': '#000000', 'priority': 42, 'name': '...",515a9899-e3d3-47b8-a336-483b2c7cbb89,"[{'begin': '20230228T062730', 'end': '20230228...",20230228T170251,515a9899-e3d3-47b8-a336-483b2c7cbb89,[{'impacted_stops': [{'amended_arrival_time': ...,515a9899-e3d3-47b8-a336-483b2c7cbb89,realtime.sncf.piv,,515a9899-e3d3-47b8-a336-483b2c7cbb89,"[{'text': 'Travaux sur les voies', 'channel': ..."
4,past,5a5103d0-592d-4315-8999-52f0bb31802d,"{'color': '#000000', 'priority': 42, 'name': '...",5a5103d0-592d-4315-8999-52f0bb31802d,"[{'begin': '20230228T012500', 'end': '20230228...",20230228T170251,5a5103d0-592d-4315-8999-52f0bb31802d,[{'impacted_stops': [{'amended_arrival_time': ...,5a5103d0-592d-4315-8999-52f0bb31802d,realtime.sncf.piv,,5a5103d0-592d-4315-8999-52f0bb31802d,NaN


## Etat du retard

In [140]:
df2 = pd.DataFrame(list(df['severity']))
df2.head(10)

,color,priority,name,effect
0,#000000,42,additional service,ADDITIONAL_SERVICE
1,#000000,42,additional service,ADDITIONAL_SERVICE
2,#000000,42,trip delayed,SIGNIFICANT_DELAYS
3,#000000,42,trip delayed,SIGNIFICANT_DELAYS
4,#000000,42,additional service,ADDITIONAL_SERVICE
5,#000000,42,trip canceled,NO_SERVICE
6,#000000,42,trip delayed,SIGNIFICANT_DELAYS
7,#000000,42,additional service,ADDITIONAL_SERVICE
8,#000000,42,trip delayed,SIGNIFICANT_DELAYS
9,#000000,42,additional service,ADDITIONAL_SERVICE


## Informations complétementaires sur le retard (ligne par ligne)

In [210]:
num_train = 13

df3 = pd.DataFrame(list(df['impacted_objects'][num_train]))
df3 = pd.DataFrame(list(df3['impacted_stops'])[0])
df3 = df3[(df3['amended_arrival_time'].notnull())]

df3 = df3[['stop_point','stop_time_effect','base_arrival_time','amended_arrival_time','cause']]
df3['retard_min'] = df3['amended_arrival_time'].astype('int32') - df3['base_arrival_time'].astype('int32')

def modif_retard(retard):
    return str(retard)[:-2]

df3['retard_min'] = df3['retard_min'].apply(modif_retard)

df3['gare'] = pd.DataFrame(list(df3['stop_point']))['name']
df3 = df3[['gare','stop_time_effect','cause','retard_min']]
df3

,gare,stop_time_effect,cause,retard_min
0,Orléans,delayed,Réutilisation d'un train,5
1,Meung-sur-Loire,delayed,Réutilisation d'un train,5
2,Beaugency,delayed,Réutilisation d'un train,45
3,Mer,delayed,Réutilisation d'un train,5
4,Blois - Chambord,delayed,Réutilisation d'un train,5
5,Onzain - Chaumont-sur-Loire,delayed,Réutilisation d'un train,5
6,Amboise,delayed,Réutilisation d'un train,5
7,Montlouis,delayed,Réutilisation d'un train,5
8,Saint-Pierre-des-Corps,delayed,Réutilisation d'un train,5
9,Tours,delayed,Réutilisation d'un train,5


## Combinaison

In [236]:
df_fin = df2.copy()
df_fin = df_fin[df_fin['effect'] == 'SIGNIFICANT_DELAYS']
df_fin = df_fin[['effect']]

In [237]:
gare_d = []
gare_a = []
cause = []
retard = []

for i in df_fin.index:
    df_mod = pd.DataFrame(list(df['impacted_objects'][i]))
    df_mod = pd.DataFrame(list(df_mod['impacted_stops'])[0])
    df_mod = df_mod[(df_mod['amended_arrival_time'].notnull())]

    df_mod = df_mod[['stop_point','stop_time_effect','base_arrival_time','amended_arrival_time','cause']]
    df_mod['retard_min'] = df_mod['amended_arrival_time'].astype('int32') - df_mod['base_arrival_time'].astype('int32')

    def modif_retard(retard):
        return str(retard)[:-2]

    df_mod['retard_min'] = df_mod['retard_min'].apply(modif_retard)

    df_mod['gare'] = pd.DataFrame(list(df_mod['stop_point']))['name']
    df_mod = df_mod[['gare','stop_time_effect','cause','retard_min']]
    
    gare_d.append(df_mod.iloc[0]['gare'])
    gare_a.append(df_mod.iloc[-1]['gare'])
    cause.append(df_mod.iloc[-1]['cause'])
    retard.append(df_mod.iloc[-1]['retard_min'])

In [238]:
df_fin['gare_depart'] = gare_d
df_fin['gare_arrivee'] = gare_a
df_fin['cause'] = cause
df_fin['retard_min'] = retard

In [239]:
df_fin

,effect,gare_depart,gare_arrivee,cause,retard_min
2,SIGNIFICANT_DELAYS,Annemasse,Évian-les-Bains,Embouteillage,5
3,SIGNIFICANT_DELAYS,Bordeaux Saint-Jean,Marseille - Saint-Charles,Travaux sur les voies,5
6,SIGNIFICANT_DELAYS,Strasbourg,Lauterbourg,Mise à quai tardive en gare origine,25
8,SIGNIFICANT_DELAYS,Colmar,Paris Est,Réutilisation d'un train,5
10,SIGNIFICANT_DELAYS,Marseille - Saint-Charles,Paris - Gare de Lyon - Hall 1 & 2,Régulation du trafic,10
13,SIGNIFICANT_DELAYS,Orléans,Tours,Réutilisation d'un train,5
14,SIGNIFICANT_DELAYS,Boulogne-Ville,Paris Nord,,5
15,SIGNIFICANT_DELAYS,Metz Ville,Sarrebourg,Incident de circulation,100
16,SIGNIFICANT_DELAYS,Forbach,Metz Ville,Réutilisation d'un train,5
17,SIGNIFICANT_DELAYS,Toulouse Matabiau,Capdenac,Prise en charge de clients en correspondance,10
